Ce notebook exécute le fichier `init_db.sql` pour créer les tables dans PostgreSQL depuis Jupyter Notebook.

Importation des bibliothèques nécessaires

In [1]:
# Ce notebook exécute le fichier `init_db.sql` pour créer les tables dans PostgreSQL depuis Jupyter Notebook.

import psycopg2

# Paramètres de connexion
DB_NAME = "stocks_db"
DB_USER = "postgres"
DB_PASSWORD = "postgres"
DB_HOST = "db"  # À utiliser depuis le container Jupyter, le service PostgreSQL est nommé "db"

# Établir la connexion à PostgreSQL
try:
    conn = psycopg2.connect(f"dbname={DB_NAME} user={DB_USER} password={DB_PASSWORD} host={DB_HOST}")
    cur = conn.cursor()
    print("✅ Connexion à PostgreSQL réussie (mode md5)")
except Exception as e:
    print(f"⚠️ Erreur lors de la connexion : {e}")
    raise

✅ Connexion à PostgreSQL réussie (mode md5)


Lecture et exécution de `init_db.sql`

In [2]:
# Lecture et exécution de `init_db.sql`
sql_file = "init_db.sql"

try:
    with open(sql_file, "r") as file:
        sql_commands = file.read().split(";")  # Séparer chaque commande SQL
except Exception as e:
    print(f"⚠️ Erreur lors de la lecture du fichier {sql_file} : {e}")
    raise

for command in sql_commands:
    command = command.strip()
    if command:  # Éviter d'exécuter des lignes vides
        try:
            cur.execute(command)
            print(f"✅ Exécuté : {command[:50]}...")
        except Exception as e:
            print(f"⚠️ Erreur ignorée lors de l'exécution de la commande : {e}")

# Commit des changements
conn.commit()
print("✅ Toutes les commandes SQL ont été exécutées avec succès.")

✅ Exécuté : DROP TABLE IF EXISTS stock_prices...
✅ Exécuté : DROP TABLE IF EXISTS stocks...
✅ Exécuté : -- Table des entreprises (stocks)
CREATE TABLE sto...
✅ Exécuté : -- Table des prix des actions (stock_prices)
CREAT...
✅ Toutes les commandes SQL ont été exécutées avec succès.


Vérification des tables créées

In [3]:
# Vérification des tables créées et de leurs champs (colonnes)
cur.execute("SELECT table_name FROM information_schema.tables WHERE table_schema = 'public';")
tables = cur.fetchall()

print("📋 Tables présentes dans la base de données et leurs champs :")
for table in tables:
    table_name = table[0]
    print(f" - {table_name}")
    # Récupérer les colonnes de la table en cours
    cur.execute(
        "SELECT column_name, data_type FROM information_schema.columns WHERE table_name = %s;",
        (table_name,)
    )
    columns = cur.fetchall()
    for column in columns:
         print(f"     - {column[0]} ({column[1]})")

📋 Tables présentes dans la base de données et leurs champs :
 - stock_prices
     - id (integer)
     - ticker (character varying)
     - date (date)
     - open (double precision)
     - high (double precision)
     - low (double precision)
     - close (double precision)
     - volume (bigint)
 - stocks
     - ticker (character varying)
     - longname (character varying)
     - shortname (character varying)
     - sector (character varying)
     - industry (character varying)
     - country (character varying)
     - fulltimeemployees (integer)
     - city (character varying)
     - state (character varying)
     - zip (character varying)
     - website (character varying)
     - phone (character varying)
     - longbusinesssummary (text)
     - exchange (character varying)
     - quotetype (character varying)
     - marketcap (bigint)
     - enterprisevalue (bigint)
     - forwardeps (double precision)
     - trailingpe (double precision)
     - dividendrate (double precision)
    

Fermeture de la connexion

In [4]:
# Fermeture de la connexion
cur.close()
conn.close()
print("✅ Connexion fermée")

✅ Connexion fermée


In [5]:
import psycopg2
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT

# Paramètres de connexion à la base "postgres" (base de maintenance)
db_params = {
    "host": "db",
    "database": "postgres",
    "user": "postgres",
    "password": "postgres"
}

DB_NAME = "user_db"

def database_exists(cursor, db_name):
    cursor.execute("SELECT 1 FROM pg_database WHERE datname = %s", (db_name,))
    return cursor.fetchone() is not None

try:
    connection = psycopg2.connect(**db_params)
    connection.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)
    cursor = connection.cursor()

    if not database_exists(cursor, DB_NAME):
        cursor.execute(f"CREATE DATABASE {DB_NAME};")
        print(f"La base de données '{DB_NAME}' a été créée avec succès !")
    else:
        print(f"La base de données '{DB_NAME}' existe déjà, pas de création nécessaire.")
except Exception as error:
    print(f"Erreur lors de la création de la base de données '{DB_NAME}':", error)
finally:
    if 'cursor' in locals():
        cursor.close()
    if 'connection' in locals():
        connection.close()


# --- Création de la table "users" ---
try:
    # Connexion directement sur la base "user_db"
    db_params_user = db_params.copy()
    db_params_user["database"] = DB_NAME

    connection = psycopg2.connect(**db_params_user)
    cursor = connection.cursor()

    cursor.execute(
        """
        CREATE TABLE IF NOT EXISTS users (
            id SERIAL PRIMARY KEY,
            username VARCHAR(255) NOT NULL UNIQUE,
            email VARCHAR(255) NOT NULL UNIQUE,
            password VARCHAR(255) NOT NULL
        );
        """
    )
    connection.commit()
    print("La table 'users' a été créée (ou existe déjà) dans 'user_db'!")
except Exception as error:
    print("Erreur lors de la création de la table 'users' :", error)
finally:
    if 'cursor' in locals():
        cursor.close()
    if 'connection' in locals():
        connection.close()


La base de données 'user_db' a été créée avec succès !
La table 'users' a été créée (ou existe déjà) dans 'user_db'!
